In [1]:
from rxitect.generator.models.smiles_transformer import SmilesTransformer
from rxitect.generator.models.components.transformer_encoder import TransformerEncoder
from rxitect.generator.datamodules.chembl_datamodule import ChemblSmilesDataModule
from rxitect.structs.vocabulary import SmilesVocabulary
from rxitect.generator.datamodules.components.molecule_dataset import MoleculeDataset
from torch.utils.data import DataLoader

import pytorch_lightning as pl
import matplotlib.pyplot as plt
import numpy as np
import dask.dataframe as dd

/home/julius/miniconda3/envs/rxt-dev/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
data_dir = "../data/"
chembl_path = "../data/processed/smiles_chembl_corpus.tsv"
vocabulary_filepath = "../data/processed/smiles_voc.txt"
dm = ChemblSmilesDataModule(data_dir, chembl_path, vocabulary_filepath=vocabulary_filepath)

In [3]:
net = TransformerEncoder(85)

In [4]:
model = SmilesTransformer(net)

In [5]:
# data = dd.read_table("../data/processed/smiles_chembl_corpus.tsv").head(n=1000)
# ds = MoleculeDataset(data=data, vocabulary=vocabulary)
# dl = DataLoader(ds, batch_size=256, pin_memory=True)
# test_batch = next(iter(dl))
# test_batch

In [6]:
# out = model.forward(test_batch)
# out.size()

In [7]:
# plt.matshow(out[:,0,:].detach())

In [8]:
trainer = pl.Trainer(gpus=1, fast_dev_run=True)

/home/julius/miniconda3/envs/rxt-dev/lib/python3.8/site-packages/pytorch_lightning/loops/utilities.py:91: PossibleUserWarning: `max_epochs` was not set. Setting it to 1000 epochs. To train without an epoch limit, set `max_epochs=-1`.
  rank_zero_warn(
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
Running in fast_dev_run mode: will run a full train, val, test and prediction loop using 1 batch(es).
`Trainer(limit_train_batches=1)` was configured so 1 batch per epoch will be used.
`Trainer(limit_val_batches=1)` was configured so 1 batch will be used.
`Trainer(limit_test_batches=1)` was configured so 1 batch will be used.
`Trainer(limit_predict_batches=1)` was configured so 1 batch will be used.
`Trainer(val_check_interval=1.0)` was configured so validation will run at the end of the training epoch..


In [9]:
trainer.fit(model=model, datamodule=dm)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


ValueError: Expected float between 0 and 1 pct_start, but got 6.0

In [ ]:
dm.train_dataloader().__dict__

In [ ]:
model.

In [ ]:
70_000 / 256

In [ ]:
trainer.max_epochs

In [10]:
import pytorch_lightning

In [11]:
pytorch_lightning.callbacks.LearningRateMonitor

pytorch_lightning.callbacks.lr_monitor.LearningRateMonitor